In [63]:
import numpy as np 
import pandas as pd 
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marlonfacey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
reviews=pd.read_csv("austin_reviews.csv")

In [203]:
reviews
reviews = reviews.dropna(axis=0, subset=['comments'])

In [204]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,Polarity,Subjectivity
0,2265,963,2009-03-17,7538,Niall,I stayed here during SXSW and had a really ple...,0.352083,0.466667
1,2265,1057,2009-03-22,10029,Michael,"Great place, close enough to everything downto...",0.307407,0.653704
2,2265,200418,2011-03-16,61677,Gustaf,We had a great time in Austin staying at Paddy...,0.255333,0.432000
3,2265,1001630,2012-03-15,1523753,Noah,We had a great stay at Zen East for South By S...,0.399150,0.745578
4,2265,1016390,2012-03-19,1547660,Melissa,I arrived late in the evening so did not meet ...,0.369444,0.681481
...,...,...,...,...,...,...,...,...
324568,47385871,721972440,2021-01-04,90722031,RYan,College area. but quick access to everything,0.333333,0.500000
324569,47411972,723017845,2021-01-09,125176457,Paul,Really loved staying here— I’m an Austin nativ...,0.367961,0.605795
324570,47411972,724050419,2021-01-13,280491070,Shanté,Kylen’s place was great! Communication was a b...,0.541667,0.783333
324571,47430860,723717982,2021-01-11,136891080,Roderick,I’ve been using Airbnb for more years with ple...,0.385897,0.557692


In [195]:
from textblob import TextBlob
temp = list(map(lambda x: TextBlob(x).sentiment, reviews['comments'].astype(str)))
reviews['Polarity'] = list(map(lambda x: x.polarity, temp))
reviews['Subjectivity'] = list(map(lambda x: x.subjectivity, temp))

/Users/marlonfacey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/marlonfacey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [234]:
revs_class = reviews[:100000]['Polarity'] < 0
neg_revs_idx = [i for i, x in enumerate(revs_class) if x]
neg_revs = [reviews.iloc[i] for i in neg_revs_idx]


In [236]:
for i in range(len(neg_revs[:5])):
    print(neg_revs[i]['comments'] + '\n')

Paddy was extremely responsive and accommodating. We would definitely stay again!

I can't say anything about the hospitality, as I never made it down to Texas.  I missed my flight due to winter weather conditions.  Sylvia didn't refund any of the cost.  

the house was a good value. the weather was crazy cold which had nothing to do with the host but it did color our experience as the house was freezing cold and nothing seemed to help. we used a space heater and the house had electrical problems so that worked occasionally. the internet was down 5 of 6 days so it was very difficult to do business. 
hosts were responsive and helpful. 

My teenage daughter, Kailey, and I have always stayed at hotels but this time we decided to try airbnd and save some money on hotel accomodations. Kailey initially though it was a bad idea to stay in a "stranger's" home and she thought it would feel "creepy." But, after we walked into the home and met Elizabeth and David we both felt "at home." They had 

In [38]:
len(reviews)

324348

In [193]:
for review in reviews['comments'][300010:300015]:
    print(review + "\n")

Cute, industrial loft! Close by a lot of restaurants and things to do downtown (although I wouldn't say it's in a walkable location...while the building felt secure, there were a lot of homeless people right down the street). Building is not very sound insulated, so you definitely hear a lot of street noises. Comfy beds & well stocked with the necessities!

We loved Paul and Greta’s place! It was perfect for what we needed for a short Austin getaway. The in-unit laundry was extremely helpful. The on-site secure parking lot was also perfect as we could take short driving trips to meals or sight-see around Austin without worrying about finding/paying for parking when we returned. The 2nd bedroom would make this an ideal unit for a small family with kids to stay in.

Place was great.

Place is amazing close to everything! Will def be staying here again! Happy I picked this location over others in the area!

This place was really cute, comfy and clean, in a location that made it easy to ge

In [18]:
# Read more data in
# import glob

# all_files = glob.glob("austin_reviews_2021_2019"+"/*.csv")
# li = []
# all_files[0]
# for filename in all_files:
#     df = pd.read_csv(filename)
#     li.append(df)
# listings_all = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
# Doc2Vec Article: https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/
# K-means Clustering: https://towardsdatascience.com/unsupervised-sentiment-analysis-a38bf1906483
# BERT Explanation: https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270

In [136]:
tokenized_reviews = []
for review in reviews['comments'][:100000]:
    tokenized_reviews.append(word_tokenize(review.lower()))

tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_reviews)]

In [137]:
d2v_model = Doc2Vec(tagged_data, vector_size = 256, window = 5, min_count = 1, epochs = 30)

In [138]:
# See similar reviews to the first review
test_doc = word_tokenize(reviews['comments'].iloc[0].lower())
test_doc_vector = d2v_model.infer_vector(test_doc)
closest_rev_idx = [i[0] for i in d2v_model.dv.most_similar(positive = [test_doc_vector])]
d2v_model.dv.most_similar(positive = [test_doc_vector])

[(0, 0.6475984454154968),
 (6029, 0.49589163064956665),
 (27972, 0.4786129593849182),
 (69536, 0.4771136939525604),
 (92061, 0.46034106612205505),
 (99126, 0.45918145775794983),
 (57014, 0.4540438652038574),
 (65756, 0.4497096836566925),
 (39191, 0.44685259461402893),
 (26148, 0.44654518365859985)]

In [139]:
print(reviews['comments'][0] + "\n")
for i in range(len(closest_rev_idx)):
    print(reviews['comments'][closest_rev_idx[i]] + "\n")


I stayed here during SXSW and had a really pleasant stay.  The house is a very relaxing environment and Paddy is both a friendly and professional host. 

I stayed here during SXSW and had a really pleasant stay.  The house is a very relaxing environment and Paddy is both a friendly and professional host. 

Alyson was very cordial and went above and beyond my expectations.  It's a must stay if you are in the Austin area.  Thanks Alyson!


The space is great for 1-2 travelers. I really liked the kitchen amenities and cutlery. You have all the utensils needed to prepare a meal. There's a stove-top, oven and toaster. The space is clean and efficient. One thing to keep in mind is the toilet seat is on a 1 foot high pedestal. The shower is also on an 8-inch pedestal and it's super tiny. Not a deal breaker but just something to keep in mind.

Transportation is not a problem because the bus stop is literally right outside the door. There are several restaurants about a 5 minute walk away and t

In [153]:
# Use K means cluster to get 2 clusters
k_means_model = KMeans(n_clusters=2, max_iter=1000, n_init=50).fit(X=d2v_model.dv.vectors)
positive_cluster_center = k_means_model.cluster_centers_[0]
negative_cluster_center = k_means_model.cluster_centers_[1]

In [165]:
top_clust_0_rev_idx = [i[0] for i in d2v_model.dv.similar_by_vector(k_means_model.cluster_centers_[0], topn=10, restrict_vocab=None)]
d2v_model.dv.similar_by_vector(k_means_model.cluster_centers_[0], topn=10, restrict_vocab=None)

[(86986, 0.9757398366928101),
 (33482, 0.9732855558395386),
 (15111, 0.9721434116363525),
 (30683, 0.9714634418487549),
 (67340, 0.9710773825645447),
 (49995, 0.9703632593154907),
 (70734, 0.9697659611701965),
 (82977, 0.9697076678276062),
 (79107, 0.9695924520492554),
 (77920, 0.9694435000419617)]

In [166]:
for i in range(len(top_clust_0_rev_idx)):
    print(reviews['comments'][top_clust_0_rev_idx[i]] + "\n")

The upstairs living area and upstairs bedrooms (2x) are beautifully presented, with chic, designer furniture. It's a very new town house, with new fittings etc. Dealing with the host was smooth, and her friend was available bang on time to let us in and help us check-in. However, the 3rd bedroom was not depicted - it's not really adequate other than for emergencies. The host should add photos of this third bedroom for transparency. The neighborhood is also quite dodgy. There are quite a lot of drug addicts, homeless people and what look like some street gangs congregating on nearby corners at night. I guess it depends on what your experience has been like in life, but certainly coming from Australia, this was quite an eyeopening experience for us. If you only need 2 bedrooms, and accept that the neighborhood is quite dodgy, then this would a good place to stay.

Great house for larger groups. Close to downtown.  Place was very clean and perfect for the bachelor party we were on.  

Ami

In [150]:
top_clust_1_rev_idx = [i[0] for i in d2v_model.dv.similar_by_vector(k_means_model.cluster_centers_[1], topn=10, restrict_vocab=None)]
d2v_model.dv.similar_by_vector(k_means_model.cluster_centers_[1], topn=10, restrict_vocab=None)

[(38967, 0.9822460412979126),
 (76283, 0.9759654402732849),
 (47438, 0.9753592610359192),
 (5853, 0.9746848940849304),
 (70728, 0.9745059609413147),
 (76772, 0.9743836522102356),
 (49485, 0.9743115901947021),
 (91646, 0.9739499092102051),
 (42364, 0.9736538529396057),
 (98709, 0.9735928773880005)]

In [151]:
for i in range(len(top_clust_1_rev_idx)):
    print(reviews['comments'][top_clust_1_rev_idx[i]] + "\n")

We had the absolute best time staying at this air bnb. It had everything we needed, was very clean, well stocked and decorated perfectly. It was a perfect location to everything and felt safe to walk home to. We had plenty of space there being 6 of us and the fun backyard was a great bonus as well. 

A++

Gorgeous house that style wise looks very Austin-y with nice decor. There's a separate guest house in the back is very nice with it's own bathroom, two additional bedrooms in the main house share a bathroom. House had everything we needed for a relaxing weekend trip!

Krista was an excellent host. Good comfortable detached apartment in an excellent neighborhood, within a couple minutes to downtown or south congress. Highly recommend. 

Wonderful location. Amazing backyard. Clean and well kept. Charming home perfect for a weekend getaway

The house was located in a great location! Easy to walk to any restaurant in SoCo. The secure backyard was a bonus for the toddlers we were traveling

In [152]:
cosine_similarity(k_means_model.cluster_centers_[0].reshape(1,-1), k_means_model.cluster_centers_[1].reshape(1,-1))

array([[0.9396877]], dtype=float32)

In [ ]:
review_scores = pd.DataFrame(d2v_model.dv.vocab.keys())
review_scores.columns = ['reviews']
# review_scores['vectors'] = review_scores.words.apply(lambda x: word_vectors.wv[f'{x}'])
# review_scroes['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
# review_scores.cluster = words.cluster.apply(lambda x: x[0])
# review_scores['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
# review_scores['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
# review_scores['sentiment_coeff'] = words.closeness_score * words.cluster_value
review_scores